In [1]:
from logging import getLogger
from logging.config import dictConfig

from torch.utils.data import DataLoader
from torcheval.metrics import BinaryAUROC
from yaml import safe_load

from streamvigil import ADBenchDataset
from streamvigil.detectors import BasicAutoEncoder, BasicDetector

In [3]:
# Logger
with open("./logging.yml", encoding="utf-8") as file:
    config = safe_load(file)
dictConfig(config)
logger = getLogger(__name__)

In [4]:
# Anomaly Detector
auto_encoder = BasicAutoEncoder(
    encoder_dims=[500, 450, 400, 350, 300, 250],
    decoder_dims=[250, 300, 350, 400, 450, 500],
    batch_norm=True,
)
detector = BasicDetector(auto_encoder)

In [6]:
random_state = 42

# Load dataset
train_data = ADBenchDataset(
    "../data/9_census.npz",
    train=True,
    random_state=random_state,
)
test_data = ADBenchDataset(
    "../data/9_census.npz",
    train=False,
    random_state=random_state,
)

# DataLoader
train_loader = DataLoader(train_data, batch_size=512)
test_loader = DataLoader(test_data, batch_size=256)

In [7]:
# Training
epochs = 10
logger.info("Start training the model...")
for epoch in range(epochs):
    logger.info(f"Epoch: {epoch+1}")
    for batch, (X, _) in enumerate(train_loader):
        loss = detector.train(X)

        if batch % 100 == 0:
            logger.info(f"Loss: {loss.item():>7f}")
logger.info("Completed training the model!")

2024-04-24 22:18:33,144 [INFO] __main__: Start training the model...
2024-04-24 22:18:33,145 [INFO] __main__: Epoch: 1
2024-04-24 22:18:33,449 [INFO] __main__: Loss: 0.229919
2024-04-24 22:18:34,958 [INFO] __main__: Loss: 0.014244
2024-04-24 22:18:36,509 [INFO] __main__: Loss: 0.011419
2024-04-24 22:18:38,036 [INFO] __main__: Loss: 0.010067
2024-04-24 22:18:39,493 [INFO] __main__: Loss: 0.009719
2024-04-24 22:18:39,621 [INFO] __main__: Epoch: 2
2024-04-24 22:18:39,636 [INFO] __main__: Loss: 0.009923
2024-04-24 22:18:41,163 [INFO] __main__: Loss: 0.009176
2024-04-24 22:18:42,755 [INFO] __main__: Loss: 0.007920
2024-04-24 22:18:44,208 [INFO] __main__: Loss: 0.007418
2024-04-24 22:18:45,716 [INFO] __main__: Loss: 0.007440
2024-04-24 22:18:45,839 [INFO] __main__: Epoch: 3
2024-04-24 22:18:45,855 [INFO] __main__: Loss: 0.007788
2024-04-24 22:18:47,299 [INFO] __main__: Loss: 0.007400
2024-04-24 22:18:48,757 [INFO] __main__: Loss: 0.006511
2024-04-24 22:18:50,220 [INFO] __main__: Loss: 0.0062

In [8]:
# Evaluation
metrics = BinaryAUROC()
logger.info("Start evaluating the model...")

for X, y in test_loader:
    scores = detector.predict(X)
    metrics.update(scores, y)

# Compute AUROC score   
logger.info(f"AUROC Score: {metrics.compute()}")

logger.info("Completed the evaluation of the model!")

2024-04-24 22:20:12,321 [INFO] __main__: Start evaluating the model...
2024-04-24 22:20:13,573 [INFO] __main__: AUROC Score: 0.6433006437574451
2024-04-24 22:20:13,574 [INFO] __main__: Completed the evaluation of the model!
